In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import random
from pathlib import Path
import json

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import gurobipy as gp
from gurobipy import GRB

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv("data_preprocessed_(19-25).csv")
data.shape
data['DecisiontoTreatDate'] = pd.to_datetime(data['DecisiontoTreatDatetime'], errors = 'coerce')
data.shape

(35273, 84)

#### For now, to make the problem easier, we only select a subset of surgeons for consideration

In [6]:
data_24_25 = data[
    (data['ScheduledDate'] >= '2024-07-01') &
    (data['ScheduledDate'] < '2025-01-01')
].copy()

list_24_25 = data_24_25['surgeonID'].value_counts().index.tolist()

counts = data[data['surgeonID'].isin(list_24_25)]['surgeonID'].value_counts()
surgeons_list = counts[counts > 100].index.tolist()
data =  data[data['surgeonID'].isin(surgeons_list)]
print(len(surgeons_list))

29


#### Let use build some number of reward classes, rewward class indexed by (Specialty)

In [7]:
n_classes = 15
ratio = data['Weighted_Case'] / data['book_dur']
r_pct = ratio.rank(method='first', pct=True)              # unique in (0,1]
data['Reward_Group'] = np.ceil(r_pct * n_classes)         # 1..n_classes
data['Reward_Group'] = data['Reward_Group'].clip(1, n_classes).astype(int)

# Everything else stays the same
data['reward_value'] = ratio
group_means = (
    data.groupby('Reward_Group', as_index=False)['reward_value']
        .mean()
        .rename(columns={'reward_value': 'mean_reward'})
)
reward_lookup = dict(zip(group_means['Reward_Group'], group_means['mean_reward']))

In [8]:
per_surgeon_counts = (
    data.dropna(subset=['Reward_Group'])
        .groupby('surgeonID')['Reward_Group']
        .nunique()
        .rename('n_distinct_groups')
        .reset_index()
)

per_surgeon_lists = (
    data.dropna(subset=['Reward_Group'])
        .groupby('surgeonID')['Reward_Group']
        .apply(lambda x: sorted(x.unique().tolist()))
        .rename('groups_present')
        .reset_index()
)

per_surgeon_summary = per_surgeon_counts.merge(per_surgeon_lists, on='surgeonID')

In [9]:
# specialty loop_up
mode_cat = (
    data.groupby('surgeonID')['SERVICE_CATEGORY']
        .agg(lambda x: x.mode().iloc[0])
)
specialty_lookup = dict(mode_cat)

In [10]:
# Duration calculation:
specialties = sorted(data['SERVICE_CATEGORY'].unique().tolist())
groups = sorted(data['Reward_Group'].unique().tolist()) 
pivot = (data
         .groupby(['SERVICE_CATEGORY', 'Reward_Group'])['book_dur']
         .mean()
         .unstack('Reward_Group'))

global_mean = data['book_dur'].mean()
pivot = pivot.apply(lambda col: col.fillna(col.mean()), axis=0)
pivot = pivot.apply(lambda row: row.fillna(row.mean()), axis=1)
pivot = pivot.fillna(global_mean)
pivot = pivot.reindex(index=specialties, columns=groups, fill_value=global_mean)
tau_nom_sc = pivot.to_numpy() 

### Constructing SAA scenarios

In [11]:
def build_arrival_scenarios_by_surgeon(
    training_raw: pd.DataFrame,
    *,
    surgeons_list,
    surgeon_col: str = "surgeonID",
    date_col: str = "DecisiontoTreatDate",
    reward_col: str = "Reward_Group",
    n_scenarios: int = 20,
    T: int = 8,
    week_anchor: str = "W-MON",
    C: int | None = None,
    random_state: int | None = 42
):
    rng = np.random.default_rng(random_state)
    df = training_raw.copy()
    df["week_start"] = df[date_col].dt.to_period(week_anchor).dt.start_time

    weekly_wide = (
        df.groupby([surgeon_col, "week_start", reward_col])
          .size().rename("arrivals").reset_index()
          .pivot_table(index=[surgeon_col, "week_start"],
                       columns=reward_col,
                       values="arrivals",
                       fill_value=0)
          .sort_index()
    )
    # Surgeon index
    surgeons = surgeons_list
    N = len(surgeons)
    sid_to_idx = surgeon_to_idx
    idx_to_sid = np.asarray(surgeons)  
    # Reward classes normalization to 0..C-1
    present_classes = set(weekly_wide.columns.tolist())
        
    full_classes = list(range(1, C + 1))
    for r in full_classes:
        if r not in weekly_wide.columns:
            weekly_wide[r] = 0
    weekly_wide = weekly_wide.reindex(columns=full_classes, fill_value=0)

    # Bootstrap T weeks per surgeon, K scenarios
    K = n_scenarios
    A = np.zeros((K, N, C, T), dtype=int)
    records = []
    grouped = dict(tuple(weekly_wide.groupby(level=0)))
    for k in range(K):
        for sid in surgeons:
            sidx = sid_to_idx[sid]
            if sid in grouped:
                block = grouped[sid]
                X = block.droplevel(0).to_numpy(int)
                n_hist = X.shape[0]
                if n_hist > 0:
                    idx = rng.integers(0, n_hist, size=T)
                    sampled = X[idx, :]
                else:
                    sampled = np.zeros((T, C), int)
            else:
                sampled = np.zeros((T, C), int)

            A[k, sidx, :, :] = sampled.T
            for t in range(T):
                for c in range(C):
                    records.append({
                        "scenario": k + 1,
                        surgeon_col: sid,
                        "week": t + 1,
                        reward_col: c,
                        "arrivals": int(sampled[t, c])
                    })

    scen_long = pd.DataFrame.from_records(records)
    return scen_long, A

In [12]:
def build_backlog_individual_at_t0(
    df,
    t0,
    group_cols=('surgeonID',),                 # keep for later aggregation
    reward_col='Reward_Group',
    arrival_date_col='DecisiontoTreatDate',
    treated_date_col='ScheduledDate',            # None if unavailable
    cancel_col=None, cancel_values=None,       # e.g., {'Cancelled','NoShow'}
    feature_cols=(
        'Patient_Gender_Binary', 'DecisionYear', 'DecisionMonth', 'DecisionDay',
        'DecisionWeekday', 'PatientType_encoded', 'Case Mix Group_encoded',
        'Diagnosis1_encoded', 'ProcedureMnemonic_encoded'),
):
    cols = list(group_cols) + [reward_col, arrival_date_col] + list(feature_cols)
    if treated_date_col and treated_date_col not in cols:
        cols.append(treated_date_col)
    if cancel_col and cancel_col not in cols:
        cols.append(cancel_col)

    use = df[cols].copy()
    use[arrival_date_col] = pd.to_datetime(use[arrival_date_col], errors='coerce')
    if treated_date_col:
        use[treated_date_col] = pd.to_datetime(use[treated_date_col], errors='coerce')

    # "Waiting at t0": arrived on/before t0, and not treated by t0, and not cancelled
    mask = use[arrival_date_col].le(t0)
    if treated_date_col:
        mask &= (use[treated_date_col].isna() | use[treated_date_col].gt(t0))
    if cancel_col and cancel_values:
        mask &= ~use[cancel_col].isin(cancel_values)

    cases = use.loc[mask].dropna(subset=list(group_cols) + [reward_col])
    return cases.reset_index(drop=True)

def backlog_counts_at_t0(cases_t0, group_cols=('surgeonID',), reward_col='Reward_Group', n_rewards=20):
    tbl = (cases_t0.groupby(list(group_cols) + [reward_col])
                  .size()
                  .unstack(fill_value=0))
    for r in range(1, n_rewards + 1):
        if r not in tbl.columns:
            tbl[r] = 0
    tbl = tbl[[r for r in range(1, n_rewards + 1)]]
    tbl.columns = [f'y_r{r}' for r in range(1, n_rewards + 1)]
    tbl['Total_Waiting'] = tbl.sum(axis=1)
    return tbl.reset_index()

def counts_df_to_dict(counts_df, group_col='surgeonID', n_rewards=20, keep_zeros=False):
    d = {}
    for _, row in counts_df.iterrows():
        sid = row[group_col]
        for r in range(1, n_rewards+1):
            cnt = int(row[f'y_r{r}'])
            if keep_zeros or cnt > 0:
                d[(sid, r)] = cnt
    return d

### Building Initial Waitlists Samples, these are ones with feature informations

#### Constructing the Prediction model

#### Implementing the gradient descent algorithm

In [14]:
C = 15
r_np = np.array([reward_lookup[c] for c in range(1, C+1)], dtype=float)
spec_list = sorted(mode_cat.unique().tolist())
S = len(spec_list)
spec_to_idx = {s:i for i,s in enumerate(spec_list)}
data["SERVICE_CATEGORY_encoded"] = data["SERVICE_CATEGORY"].map(spec_to_idx)
U_np = 420.0 * np.ones(S, dtype=float)
B =  round(S * 1.0)

In [15]:
start_date = pd.Timestamp("2023-03-01")
end_date   = pd.Timestamp("2024-03-01")

month_starts = []
current = start_date
while current <= end_date:
    month_starts.append(current)
    current += relativedelta(months=1)

In [16]:
scenario_cases   = {}
scenario_counts  = {}

for t0 in month_starts:
    # 1) individuals waiting at t0 (handles NaT or SurgeryDate > t0)
    cases_t0 = build_backlog_individual_at_t0(
        data,
        t0=t0,
        group_cols=('surgeonID',),
        reward_col='Reward_Group',
        arrival_date_col='DecisiontoTreatDate',
        treated_date_col='SurgeryDate',
        cancel_col=None, cancel_values=None,
        feature_cols=('Patient_Gender_Binary','DecisionYear','DecisionMonth','DecisionDay',
                      'DecisionWeekday','PatientType_encoded','Case Mix Group_encoded', 'Case Mix Age Category_encoded',
                     'Diagnosis1_encoded','ProcedureMnemonic_encoded','SERVICE_CATEGORY_encoded')
    )
    scenario_cases[t0] = cases_t0
    counts_t0 = backlog_counts_at_t0(
        cases_t0,
        group_cols=('surgeonID',),
        reward_col='Reward_Group',
        n_rewards=15
    )
    scenario_counts[t0] = counts_t0

In [17]:
spec_of_surgeon = np.array([spec_to_idx[mode_cat.loc[sid]] for sid in surgeons_list], dtype=int)
surgeon_to_idx = {sid: i for i, sid in enumerate(surgeons_list)}
N = len(surgeons_list)
spec_of_surgeon_np = np.array(
    [spec_to_idx.get(mode_cat.get(sid, np.nan), -1) for sid in surgeons_list],
    dtype=int
)

G_sn_np = np.zeros((S, N), dtype=float)
valid = spec_of_surgeon_np >= 0
G_sn_np[spec_of_surgeon_np[valid], np.arange(N)[valid]] = 1.0

In [18]:
K = 2
T = 8
training_raw =  data[(data['DecisiontoTreatDate'] >= pd.Timestamp("2023-05-01")) & (data['DecisiontoTreatDate'] <= pd.Timestamp("2024-03-01"))].copy()
scen_long, A_scenarios= build_arrival_scenarios_by_surgeon(
    training_raw=training_raw, surgeons_list= surgeons_list,
    n_scenarios=K, T=T, week_anchor="W-MON", C=C, random_state=2
)

In [19]:
NUM_COLS = ['DecisionYear','DecisionMonth','DecisionDay','DecisionWeekday']
CAT_COLS = ['PatientType_encoded','Case Mix Group_encoded','Patient_Gender_Binary', 
            'Case Mix Age Category_encoded', 'Diagnosis1_encoded','SERVICE_CATEGORY_encoded']
DATE_COLS = ['DecisiontoTreatDate', 'SurgeryDate']

def fit_design_plain(df):
    cat_levels  = {c: sorted(df[c].dropna().unique().tolist()) for c in CAT_COLS}

    def transform(df_):
        df_ = df_.copy()
        df_.drop(columns=[c for c in DATE_COLS if c in df_.columns], errors='ignore', inplace=True)
        dt_cols = df_.select_dtypes(include=['datetime64[ns]', 'datetimetz']).columns
        if len(dt_cols):
            df_.drop(columns=list(dt_cols), inplace=True)
        for c in CAT_COLS:
            df_[c] = pd.Categorical(df_[c], categories=cat_levels[c])

        X_num  = df_[NUM_COLS].apply(pd.to_numeric, errors='coerce').astype('float64')
        X_cat  = pd.get_dummies(df_[CAT_COLS], drop_first=True, dtype='float64')

        X = pd.concat([pd.Series(1.0, index=df_.index, name='intercept', dtype='float64'),
                       X_num, X_cat], axis=1)
        X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0).astype('float64')
        return X

    X_fit = transform(df)
    col_order = X_fit.columns.tolist()

    def transform_fixed(df_):
        return transform(df_).reindex(columns=col_order, fill_value=0.0).astype('float64')

    return transform_fixed, col_order

transform_fixed, design_cols = fit_design_plain(training_raw)  # or fit_design_no_spec_interactions
dim = len(design_cols)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LinearSoftmaxMNL(nn.Module):
    def __init__(self, d: int, C: int, *, bias: bool = False, init_std: float = 0.05, temperature: float = 1.0):
        super().__init__()
        self.W = nn.Parameter(torch.empty(C, d))
        self.b = nn.Parameter(torch.zeros(C)) if bias else None
        nn.init.normal_(self.W, std=init_std)   # small init to avoid peaky softmax
        self.tau = float(temperature)

    def forward(self, X: torch.Tensor, return_logits: bool = False):
        logits = X @ self.W.T
        if self.b is not None:
            logits = logits + self.b
        if return_logits:
            return logits
        return F.softmax(logits / self.tau, dim=1)

    @torch.no_grad()
    def project_params(self):
        pass  # keep for compatibility with your training loop


In [21]:
def make_first_stage_layer_SAA_arrivals(
    N, S, C, T, Kb,
    spec_of_surgeon_np, U_s_np, r_per_min_c_np, G_sn_np,
    A_scenarios_np,               # <-- (Kb, N, C, T), fixed constants
    beta_sc_np=None, rho=1e-3,
    tau_nom_sc=None, Ymax=25, surgeon_cap=1800
):
    N=int(N); S=int(S); C=int(C); T=int(T); Kb=int(Kb)
    spec = np.asarray(spec_of_surgeon_np, int).reshape(N)
    U_s  = np.asarray(U_s_np, float).reshape(S)
    r_c  = np.asarray(r_per_min_c_np, float).reshape(C)
    G_sn = np.asarray(G_sn_np, float).reshape(S, N)
    A_it = np.full((N, T), float(surgeon_cap), dtype=float)
    if tau_nom_sc is None:
        tau_sc = np.full((S, C), 100.0, dtype=float)
        tau_sct = np.repeat(tau_sc[:, :, None], T, axis=2)
    else:
        arr = np.asarray(tau_nom_sc, float)
        tau_sct = np.repeat(arr[:, :, None], T, axis=2) if arr.ndim==2 else arr
    tau_ict  = tau_sct[spec, :, :]
    rEff_ict = r_c[None, :, None] * tau_ict

    BetaIc = None
    if beta_sc_np is not None:
        BetaIc = np.asarray(beta_sc_np, float).reshape(S, C)[spec, :]

    # constants
    UcS    = cp.Constant(U_s)
    Gc     = cp.Constant(G_sn)
    TauIcC = cp.Constant(tau_ict)
    rEffC  = cp.Constant(rEff_ict)
    AitC   = cp.Constant(A_it)
    Aconst = cp.Constant(np.asarray(A_scenarios_np, float).reshape(Kb, N, C, T))
    BetaIcC= (cp.Constant(BetaIc) if BetaIc is not None else None)

    # parameter
    p0 = cp.Parameter((N, C), nonneg=True)

    # decisions
    Y  = cp.Variable((S, T), nonneg=True)                 # shared
    Xs = [cp.Variable((N, C, T), nonneg=True) for _ in range(Kb)]
    Bs = [cp.Variable((N, C, T+1), nonneg=True) for _ in range(Kb)]

    cons = []
    if Ymax is not None:
        for t in range(T):
            cons += [cp.sum(Y[:, t]) <= float(Ymax)]     # only once, not per scenario

    obj_terms = []
    for k in range(Kb):
        Xk, Bk = Xs[k], Bs[k]
        cons += [Bk[:, :, 0] == p0]
        for t in range(T):
            cons += [Bk[:, :, t+1] == Bk[:, :, t] + Aconst[k, :, :, t] - Xk[:, :, t]]
            minutes_i_t = cp.sum(cp.multiply(TauIcC[:, :, t], Xk[:, :, t]), axis=1)
            cons += [minutes_i_t <= AitC[:, t]]
            cons += [(Gc @ minutes_i_t) <= cp.multiply(UcS, Y[:, t])]

        reward_k = cp.sum(cp.multiply(rEffC, Xk))
        penalty_k = 0.0
        penalty_k = cp.sum(cp.multiply(BetaIcC, cp.sum(cp.square(Bk[:, :, 1:]), axis=2)))
        obj_terms.append(reward_k - penalty_k)
        
    regY = (rho/3.0) * cp.sum_squares(Y)
    obj  = (1.0 / Kb) * cp.sum(obj_terms) - regY
    prob = cp.Problem(cp.Maximize(obj), cons)
    sm = prob.size_metrics
    print("vars:", sm.num_scalar_variables)
    layer = CvxpyLayer(prob, parameters=[p0], variables=[Y])

    def solve_layer(p0_torch):
        outs = layer(p0_torch)
        Y_t = outs[0]
        X_list = outs[1:]
        return Y_t, X_list

    return solve_layer

In [22]:
beta_np = np.full((S, C), 0.01, dtype=float)  
solve_first_stage = make_first_stage_layer_SAA_arrivals(
    N, S, C, T, K,
    spec_of_surgeon_np=spec_of_surgeon,
    U_s_np=U_np, r_per_min_c_np=r_np,
    G_sn_np=G_sn_np, A_scenarios_np= A_scenarios,
    beta_sc_np=beta_np, tau_nom_sc=tau_nom_sc,
    rho=1e-3, Ymax=25, surgeon_cap=1800
)

vars: 14870


In [23]:
def make_second_stage_value_and_grad_SAA_arrivals(
    N, S, C, T, Kb,
    spec_of_surgeon_np,        # (N,) int -> specialty index in {0,...,S-1}
    U_s_np,                    # (S,) minutes per unit of Y for each specialty
    r_per_min_c_np,            # (C,) reward per minute
    G_sn_np,                   # (S,N) indicator: G[s,i]=1 if surgeon i in specialty s
    A_scenarios_np,            # (Kb, N, C, T) fixed arrival paths (constants)
    beta_sc_np=None,           # (S,C) nonneg backlog penalty weights (or None)
    tau_nom_sc=None,           # (S,C) or (S,C,T); default 100
    surgeon_cap=1800,
    penalty_norm="L2",         # "L1" (linear, memory-light) or "L2" (quadratic)
):

    N=int(N); S=int(S); C=int(C); T=int(T); Kb=int(Kb)
    spec = np.asarray(spec_of_surgeon_np, int).reshape(N)
    U_s  = np.asarray(U_s_np,         float).reshape(S)
    r_c  = np.asarray(r_per_min_c_np, float).reshape(C)
    G_sn = np.asarray(G_sn_np,        float).reshape(S, N)
    A_it = np.full((N, T), float(surgeon_cap), dtype=float)

    # tau -> (S,C,T) then map to surgeons (N,C,T)
    if tau_nom_sc is None:
        tau_sc  = np.full((S, C), 100.0, float)
        tau_sct = np.repeat(tau_sc[:, :, None], T, axis=2)
    else:
        arr = np.asarray(tau_nom_sc, float)
        tau_sct = np.repeat(arr[:, :, None], T, axis=2) if arr.ndim==2 else arr
    tau_ict  = tau_sct[spec, :, :]                  # (N,C,T)
    r_eff_ict = r_c[None, :, None] * tau_ict        # (N,C,T)

    BetaIc = None
    if beta_sc_np is not None:
        BetaIc = np.asarray(beta_sc_np, float).reshape(S, C)[spec, :]  # (N,C)

    # Constants
    UcS    = cp.Constant(U_s)
    Gc     = cp.Constant(G_sn)
    TauIcC = cp.Constant(tau_ict)
    rEffC  = cp.Constant(r_eff_ict)
    AitC   = cp.Constant(A_it)
    Aconst = cp.Constant(np.asarray(A_scenarios_np, float).reshape(Kb, N, C, T))
    BetaIcC= (cp.Constant(BetaIc) if BetaIc is not None else None)

    # Parameters (inputs to the recourse)
    Yp = cp.Parameter((S, T), nonneg=True)      # first-stage blocks (shared across scenarios)
    p0 = cp.Parameter((N, C), nonneg=True)      # initial backlog

    # Decisions per scenario
    Xs = [cp.Variable((N, C, T),  nonneg=True) for _ in range(Kb)]
    Bs = [cp.Variable((N, C, T+1), nonneg=True) for _ in range(Kb)]

    cons = []
    # Track the specialty-capacity constraints per scenario/week to read duals later
    cap_cons = [[None for _ in range(T)] for _ in range(Kb)]

    for k in range(Kb):
        Xk, Bk = Xs[k], Bs[k]
        cons += [Bk[:, :, 0] == p0]
        for t in range(T):
            cons += [Bk[:, :, t+1] == Bk[:, :, t] + Aconst[k, :, :, t] - Xk[:, :, t]]
            minutes_i_t = cp.sum(cp.multiply(TauIcC[:, :, t], Xk[:, :, t]), axis=1)  # (N,)
            cons += [minutes_i_t <= AitC[:, t]]

            # Specialty capacity tied to Yp (non-anticipative)
            cap = (Gc @ minutes_i_t) <= cp.multiply(UcS, Yp[:, t])                  # (S,)
            cons.append(cap)
            cap_cons[k][t] = cap

    obj_terms = []
    for k in range(Kb):
        Xk, Bk = Xs[k], Bs[k]
        reward_k = cp.sum(cp.multiply(rEffC, Xk))
        if BetaIcC is None:
            pen_k = 0.0
        else:
            if penalty_norm.upper() == "L2":
                # quadratic per (i,c) on entire trajectory (QP; may be heavier)
                pen_k = cp.sum(cp.multiply(BetaIcC, cp.sum(cp.square(Bk[:, :, 1:]), axis=2)))
            else:
                # default: linear (memory-friendly)
                pen_k = cp.sum(cp.multiply(BetaIcC, cp.sum(Bk[:, :, 1:], axis=2)))
        obj_terms.append(reward_k - pen_k)

    obj = cp.Maximize((1.0 / Kb) * cp.sum(obj_terms))  # sample average
    prob = cp.Problem(obj, cons)

    def solve(Y_val, p0_val, *, solver=cp.GUROBI, **solver_opts):
        Yp.value = np.maximum(np.asarray(Y_val,  float).reshape(S, T), 0.0)
        p0.value = np.maximum(np.asarray(p0_val, float).reshape(N, C), 0.0)
        prob.solve(solver=solver, **solver_opts)
        if prob.status not in ("optimal", "optimal_inaccurate"):
            raise RuntimeError(f"Second-stage not optimal: {prob.status}")

        v_avg = float(prob.value)  # already averaged by (1/Kb)
        Pi_avg = np.zeros((S, T), dtype=float)
        for t in range(T):
            acc = np.zeros(S, dtype=float)
            for k in range(Kb):
                acc += cap_cons[k][t].dual_value
            Pi_avg[:, t] = acc
        gradY = U_s[:, None] * Pi_avg

        return v_avg, gradY

    return solve

In [24]:
# assumes you've already defined: S, C, T, U_np, r_per_min_c_np (shape (C,)), beta_np, tau_nom_np
second_stage = make_second_stage_value_and_grad_SAA_arrivals(
    N=N, S=S, C=C, T=T, Kb=K,
    spec_of_surgeon_np = spec_of_surgeon_np,
    U_s_np=U_np,
    r_per_min_c_np=r_np, 
    G_sn_np = G_sn_np,
    A_scenarios_np= A_scenarios,
    beta_sc_np=beta_np,
    tau_nom_sc=tau_nom_sc,
    penalty_norm="L2"
)

In [27]:
def to_surgeon_idx(series):
    """Map a pandas Series of surgeonIDs -> integer indices (may contain NaN)."""
    return series.map(surgeon_to_idx)
    
def build_p_and_d_for_t0_by_surgeon(
    cases_t0: pd.DataFrame,
    counts_t0: pd.DataFrame,
    model,
    transform_fixed,
    C: int,
    mode_cat: pd.Series | None = None,   # Series: index=surgeonID, value=SERVICE_CATEGORY
    ensure_service_category: bool = True
):

    if ensure_service_category and 'SERVICE_CATEGORY' not in cases_t0.columns:
        cases_t0 = cases_t0.merge(
            mode_cat.rename('SERVICE_CATEGORY'),
            left_on='surgeonID', right_index=True, how='left'
        )

    # --- 1) Predict per case, then aggregate to surgeon
    X_df = transform_fixed(cases_t0)
    device = torch.device("cpu")
    torch_dtype = torch.float32

    X = torch.as_tensor(X_df.to_numpy(), dtype=torch_dtype, device=device)
    P_cases = model(X)  # (n_cases, C)

    idx_series = to_surgeon_idx(cases_t0['surgeonID'])
    mask = idx_series.notna().to_numpy()
    idx_torch = torch.as_tensor(idx_series[mask].to_numpy(dtype=int), dtype=torch.long, device=device)
    P_cases_sel = P_cases[torch.as_tensor(mask, device=device)]
    p = torch.zeros((N, C), dtype=P_cases.dtype, device=device)
    p.index_add_(0, idx_torch, P_cases_sel)

    # --- 2) True counts per surgeon
    cols = [f'y_r{c}' for c in range(1, C+1)]
    for col in cols:
        if col not in counts_t0.columns:
            counts_t0[col] = 0

    d_by_surgeon = (counts_t0
                    .groupby('surgeonID', dropna=False)[cols]
                    .sum())

    # align to surgeons_list order
    d_by_surgeon = d_by_surgeon.reindex(surgeons_list, fill_value=0)
    d_true = d_by_surgeon.to_numpy(dtype=float)  # (N, C)

    return p, d_true

In [25]:
model = LinearSoftmaxMNL(dim, C)
opt = torch.optim.Adam(model.parameters(), lr=3e-3)

In [28]:
def to_cvxpy_param(x: torch.Tensor) -> torch.Tensor:
    # cvxpylayers wants CPU + torch.double; keep the graph intact
    if x.device.type != "cpu":
        x = x.cpu()
    if x.dtype != torch.double:
        x = x.double()
    return x

In [29]:
EPOCHS      = 8
BATCH_SIZE  = 4
GRAD_CLIP   = 100
month_starts_list = list(month_starts)

for epoch in range(EPOCHS):
    model.train()
    random.shuffle(month_starts_list)
    epoch_total_val = 0.0

    for b in range(0, len(month_starts_list), BATCH_SIZE):
        batch_t0s = month_starts_list[b:b+BATCH_SIZE]
        opt.zero_grad()
        loss_batch = 0.0
        val_batch  = 0.0

        for t0 in batch_t0s:
            cases_t0  = scenario_cases[t0]
            counts_t0 = scenario_counts[t0]
            p_t, d_true_np = build_p_and_d_for_t0_by_surgeon(
                cases_t0, counts_t0, model, transform_fixed, C, mode_cat=mode_cat
            )
            # p_t requires grad through model → keep graph; just cast for cvxpylayers:
            p_t_cvx = to_cvxpy_param(p_t)

            # --- 2) First stage (unchanged unpack; you keep the extra return) ---
            Y_star_t, _ = solve_first_stage(p_t_cvx)     # Y_star_t: CPU, double

            # --- 3) Second stage (numpy in/out) ---
            Y_star_np = Y_star_t.detach().cpu().numpy()
            v, gradY_np = second_stage(
                Y_val=Y_star_np,
                p0_val=d_true_np,
                solver=cp.GUROBI,
                verbose=False,
            )
            val_batch += float(v)

            # --- 4) Decision-focused loss: ⟨Y, ∂(−obj)/∂Y⟩ = ⟨Y, gradY⟩ ---
            gY = torch.as_tensor(gradY_np, dtype=Y_star_t.dtype, device=Y_star_t.device)  # (S,T)
            loss_batch = loss_batch + (Y_star_t * gY).sum()

        # average over this mini-batch (optional)
        loss_batch = loss_batch / len(batch_t0s)

        # backprop once for the batch
        loss_batch.backward()
        if GRAD_CLIP is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        opt.step()

        # your new NN has a no-op project_params(); keep the call to avoid touching other code
        with torch.no_grad():
            model.project_params()

        epoch_total_val += val_batch

    print(f"epoch {epoch+1}: sum P = {epoch_total_val:.4f}")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2721691
Academic license 2721691 - for non-commercial use only - registered to c7___@uwaterloo.ca
epoch 1: sum P = 938.7150
epoch 2: sum P = 938.7101
epoch 3: sum P = 936.8318


MemoryError: Unable to allocate 7.20 GiB for an array with shape (967026609,) and data type int64

3

In [35]:
repo_root = Path.cwd()
save_dir = repo_root / "models" / "LinearSoftmaxMNL"
save_dir.mkdir(parents=True, exist_ok=True)

# minimal config to rebuild the module
model_cfg = {
    "d_dim": int(d_dim),
    "C": int(C),
    "ref_class": int(C-1),
    "design_cols": list(design_cols),
    "clamp": {"alpha":16.0,"beta":14.0,"gamma":10.0,"delta":14.0},
}

torch.save(model.state_dict(), save_dir / "weights.pt")
with open(save_dir / "config.json", "w") as f:
    json.dump(model_cfg, f, indent=2)

In [30]:
def solve_with_SAA(
    N, S, C, T, Kb,
    spec_of_surgeon_np, U_s_np, r_per_min_c_np, G_sn_np,
    A_scenarios_np,               # shape (Kb, N, C, T)
    p0_np,                        # shape (N, C): initial backlog (d^0); nonnegative
    beta_sc_np=None,              # optional shape (S, C) for backlog penalty weights
    rho=1e-3,
    tau_nom_sc=None,              # (S, C) or (S, C, T); per-case minutes
    Ymax=25,
    surgeon_cap=1800,
    solver="GUROBI",
    verbose=False,
    **solve_kwargs
):
    """
    Returns:
        Y_val      : (S, T) optimal first-stage blocks
        X_vals     : list length Kb, each (N, C, T) case selections
        B_vals     : list length Kb, each (N, C, T+1) backlogs
        obj_value  : optimal objective value (float)
    """

    # ---- Cast & basic tensors ----
    N=int(N); S=int(S); C=int(C); T=int(T); Kb=int(Kb)

    spec = np.asarray(spec_of_surgeon_np, int).reshape(N)        # surgeon -> specialty index in [0, S)
    U_s  = np.asarray(U_s_np, float).reshape(S)                   # surgeon-minute capacity per specialty
    r_c  = np.asarray(r_per_min_c_np, float).reshape(C)           # reward per OR minute for class c
    G_sn = np.asarray(G_sn_np, float).reshape(S, N)               # S x N mapping minutes to specialties
    A_it = np.full((N, T), float(surgeon_cap), dtype=float)       # per-surgeon per-period minute cap
    A_scen = np.asarray(A_scenarios_np, float).reshape(Kb, N, C, T)
    p0_np = np.asarray(p0_np, float).reshape(N, C)

    # tau_nom_sc: (S,C) or (S,C,T) -> build (S,C,T)
    if tau_nom_sc is None:
        tau_sc = np.full((S, C), 100.0, dtype=float)
        tau_sct = np.repeat(tau_sc[:, :, None], T, axis=2)       # (S, C, T)
    else:
        arr = np.asarray(tau_nom_sc, float)
        if arr.ndim == 2:                                        # (S, C)
            tau_sct = np.repeat(arr[:, :, None], T, axis=2)
        elif arr.ndim == 3:                                      # (S, C, T)
            tau_sct = arr
        else:
            raise ValueError("tau_nom_sc must be (S,C) or (S,C,T).")

    # Map to surgeon-level (N,C,T)
    tau_ict  = tau_sct[spec, :, :]                               # (N, C, T)
    rEff_ict = r_c[None, :, None] * tau_ict                      # (N, C, T), reward per case = r_c * minutes

    # Optional backlog penalty weights Beta (N,C) from (S,C)
    BetaIc = None
    if beta_sc_np is not None:
        BetaIc = np.asarray(beta_sc_np, float).reshape(S, C)[spec, :]  # (N, C)

    # ---- Constants ----
    UcS     = cp.Constant(U_s)              # (S,)
    Gc      = cp.Constant(G_sn)             # (S, N)
    TauIcC  = cp.Constant(tau_ict)          # (N, C, T)
    rEffC   = cp.Constant(rEff_ict)         # (N, C, T)
    AitC    = cp.Constant(A_it)             # (N, T)
    Aconst  = cp.Constant(A_scen)           # (Kb, N, C, T)
    BetaIcC = (cp.Constant(BetaIc) if BetaIc is not None else None)

    # ---- Decision variables ----
    Y  = cp.Variable((S, T), nonneg=True)                    # shared across scenarios
    Xs = [cp.Variable((N, C, T), nonneg=True) for _ in range(Kb)]
    Bs = [cp.Variable((N, C, T+1), nonneg=True) for _ in range(Kb)]

    # ---- Constraints ----
    cons = []

    # Block-limit per period
    if Ymax is not None:
        for t in range(T):
            cons.append(cp.sum(Y[:, t]) <= float(Ymax))

    # Per-scenario flow & capacities
    for k in range(Kb):
        Xk, Bk = Xs[k], Bs[k]

        # Backlog flow: B_{t+1} = B_t + arrivals - service, with B_0 = p0
        cons.append(Bk[:, :, 0] == p0_np)
        for t in range(T):
            cons.append(Bk[:, :, t+1] == Bk[:, :, t] + Aconst[k, :, :, t] - Xk[:, :, t])

            # Surgeon minutes this period
            minutes_i_t = cp.sum(cp.multiply(TauIcC[:, :, t], Xk[:, :, t]), axis=1)  # (N,)
            cons.append(minutes_i_t <= AitC[:, t])                                   # per-surgeon cap
            cons.append((Gc @ minutes_i_t) <= cp.multiply(UcS, Y[:, t]))

    # ---- Objective ----
    scenario_terms = []
    for k in range(Kb):
        Xk, Bk = Xs[k], Bs[k]
        reward_k  = cp.sum(cp.multiply(rEffC, Xk))  # sum over (i,c,t)
        if BetaIcC is not None:
            # To weight per (i,c), expand Beta and sum manually:
            penalty_k = cp.sum(cp.multiply(BetaIcC, cp.sum(cp.square(Bk[:, :, 1:]), axis=2)))
        else:
            penalty_k = 0.0
        scenario_terms.append(reward_k - penalty_k)

    regY = (rho / 3.0) * cp.sum_squares(Y)
    obj  = (1.0 / Kb) * cp.sum(scenario_terms) - regY

    prob = cp.Problem(cp.Maximize(obj), cons)

    prob.solve(solver=solver, verbose=verbose, **solve_kwargs)
    Y_val  = Y.value
    X_vals = [X.value for X in Xs]
    B_vals = [B.value for B in Bs]
    return Y_val, X_vals, B_vals, prob.value

#### Genertae Y* for out of sample validations

In [32]:
# just try to rebuild the SAA sampples
K = 5
T = 8
training_raw =  data[(data['DecisiontoTreatDate'] >= pd.Timestamp("2023-05-01")) & (data['DecisiontoTreatDate'] <= pd.Timestamp("2024-03-01"))].copy()
scen_long, A_scenarios= build_arrival_scenarios_by_surgeon(
    training_raw=training_raw, surgeons_list=surgeons_list,
    n_scenarios=K, T=T, week_anchor="W-MON", C=C, random_state=2
)

In [33]:
start_date = pd.Timestamp("2024-03-01")
end_date   = pd.Timestamp("2024-08-01")

month_starts_test = []
current = start_date
while current <= end_date:
    month_starts_test.append(current)
    current += relativedelta(months=1)

In [34]:
month_starts_test

[Timestamp('2024-03-01 00:00:00'),
 Timestamp('2024-04-01 00:00:00'),
 Timestamp('2024-05-01 00:00:00'),
 Timestamp('2024-06-01 00:00:00'),
 Timestamp('2024-07-01 00:00:00'),
 Timestamp('2024-08-01 00:00:00')]

In [35]:
scenario_cases_test   = {}
scenario_counts_test  = {}
scenario_queues_test  = {}

for t0 in month_starts_test:
    # 1) individuals waiting at t0 (handles NaT or SurgeryDate > t0)
    cases_t0 = build_backlog_individual_at_t0(
        data,
        t0=t0,
        group_cols=('surgeonID',),                 # add 'SERVICE_CATEGORY' here if you want (surgeon, specialty)
        reward_col='Reward_Group',
        arrival_date_col='DecisiontoTreatDate',
        treated_date_col='SurgeryDate',
        cancel_col=None, cancel_values=None,
        feature_cols=('Patient_Gender_Binary','DecisionYear','DecisionMonth','DecisionDay',
                      'DecisionWeekday','PatientType_encoded','Case Mix Group_encoded', 'Case Mix Age Category_encoded',
                     'Diagnosis1_encoded','ProcedureMnemonic_encoded', 'SERVICE_CATEGORY_encoded')
    )
    scenario_cases_test[t0] = cases_t0
    
    counts_t0 = backlog_counts_at_t0(
        cases_t0,
        group_cols=('surgeonID',),
        reward_col='Reward_Group',
        n_rewards=15
    )
    scenario_counts_test[t0] = counts_t0
    scenario_queues_test[t0] = counts_df_to_dict(counts_t0, group_col='surgeonID', n_rewards=15, keep_zeros=False)

In [36]:
Y = {}
beta_np = np.full((S, C), 0.01, dtype=float)  

for t0 in month_starts_test:
    cases_t0 = scenario_cases_test[t0]
    counts_t0 = scenario_counts_test[t0]
    p_t, _ = build_p_and_d_for_t0_by_surgeon(
        cases_t0, counts_t0, model, transform_fixed, C, mode_cat=mode_cat
    )

    p_t_np = p_t.detach().cpu().numpy()
    Y_star, X_list, B_list, val = solve_with_SAA(
        N, S, C, T, K,
        spec_of_surgeon_np, U_s_np=U_np, r_per_min_c_np=r_np, G_sn_np=G_sn_np,
        A_scenarios_np=A_scenarios,
        p0_np=p_t_np,
        beta_sc_np=beta_np,
        rho=1e-3,
        tau_nom_sc=tau_nom_sc,
        Ymax=25,
        surgeon_cap=1800,
        verbose=True
    )
    Y[t0] = Y_star
    Y_star_np = Y_star

    t0_str = pd.Timestamp(t0).strftime("%Y-%m-%d")
    np.save(f"Y_SPO_{t0_str}.npy", Y_star_np)

(CVXPY) Nov 03 11:43:37 AM: Your problem has 37055 variables, 21143 constraints, and 0 parameters.
(CVXPY) Nov 03 11:43:37 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 03 11:43:37 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 03 11:43:37 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 03 11:43:37 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 03 11:43:37 AM: Compiling problem (target solver=GUROBI).
(CVXPY) Nov 03 11:43:37 AM: Reduction chain: FlipObjective -> CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Nov 03 11:43:37 AM: Applying reduction FlipObjective
(CVXPY) Nov 03 11:43:37 AM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 03 11:43:37 AM: Applying reduction Qp2SymbolicQp


                                     CVXPY                                     
                                     v1.7.3                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------


(CVXPY) Nov 03 11:43:41 AM: Applying reduction QpMatrixStuffing
(CVXPY) Nov 03 11:43:41 AM: Applying reduction GUROBI
(CVXPY) Nov 03 11:43:41 AM: Finished problem compilation (took 4.114e+00 seconds).
(CVXPY) Nov 03 11:43:41 AM: Invoking solver GUROBI  to obtain a solution.


-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
Set parameter OutputFlag to value 1
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
QCPDual  1

Academic license 2721691 - for non-commercial use only - registered to c7___@uwaterloo.ca
Optimize a model with 38543 rows, 54455 columns and 124455 nonzeros
Model fingerprint: 0x2d2d2686
Model has 17480 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e-02, 2e+00]
  QObjective range [7e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range    

(CVXPY) Nov 03 11:43:42 AM: Problem status: optimal
(CVXPY) Nov 03 11:43:42 AM: Optimal value: 1.376e+03
(CVXPY) Nov 03 11:43:42 AM: Compilation took 4.114e+00 seconds
(CVXPY) Nov 03 11:43:42 AM: Solver (including time spent in interface) took 3.759e-01 seconds


...

(CVXPY) Nov 03 11:43:45 AM: Your problem has 37055 variables, 21143 constraints, and 0 parameters.
(CVXPY) Nov 03 11:43:45 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 03 11:43:45 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 03 11:43:45 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 03 11:43:45 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 03 11:43:45 AM: Compiling problem (target solver=GUROBI).


-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------


(CVXPY) Nov 03 11:43:45 AM: Reduction chain: FlipObjective -> CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Nov 03 11:43:45 AM: Applying reduction FlipObjective
(CVXPY) Nov 03 11:43:45 AM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 03 11:43:45 AM: Applying reduction Qp2SymbolicQp
(CVXPY) Nov 03 11:43:45 AM: Applying reduction QpMatrixStuffing
(CVXPY) Nov 03 11:43:45 AM: Applying reduction GUROBI
(CVXPY) Nov 03 11:43:45 AM: Finished problem compilation (took 2.591e-01 seconds).
(CVXPY) Nov 03 11:43:45 AM: Invoking solver GUROBI  to obtain a solution.


-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
Set parameter OutputFlag to value 1
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
QCPDual  1

Academic license 2721691 - for non-commercial use only - registered to c7___@uwaterloo.ca
Optimize a model with 38543 rows, 54455 columns and 124455 nonzeros
Model fingerprint: 0x89654eb8
Model has 17480 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e-02, 2e+00]
  QObjective range [7e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range    

(CVXPY) Nov 03 11:43:45 AM: Problem status: optimal
(CVXPY) Nov 03 11:43:45 AM: Optimal value: 1.445e+03
(CVXPY) Nov 03 11:43:45 AM: Compilation took 2.591e-01 seconds
(CVXPY) Nov 03 11:43:45 AM: Solver (including time spent in interface) took 3.317e-01 seconds


In [36]:
Y_star.shape

(10, 8)

In [ ]:
Y = {}

for t0 in month_starts_test:
    print(t0)
    cases_t0 = scenario_cases_test[t0]
    counts_t0 = scenario_counts_test[t0]
    p_t,_ = build_p_and_d_for_t0_by_surgeon(
        cases_t0, counts_t0, model, transform_fixed, C, mode_cat=mode_cat
    )

    p_t_np = p_t.detach().cpu().numpy()
    Y_star = solve_first_stage(p_t)
    Y[t0] = Y_star
    Y_star_np = Y_star.detach().cpu().numpy()
    t0_str = pd.Timestamp(t0).strftime("%Y-%m-%d")
    np.save(f"Y_SPO_{t0_str}.npy", Y_star_np)